In [1]:
demand = [990,1980,3961,2971,1980]   
d=0  # d% shortage allowance
Y_b = [1.3086,1.3671,1.4183,1.4538,1.5122]           # Fabric yield (consumption rate) rate per garment of size 𝛽
f= 1.95 # Fabric cost 

U = 0.85
l_max= 20  #yrds
e= .083   # Fabric end allowance in yrds

if len(demand)!=len(Y_b):
    raise ValueError('number of sizes and number of fabric consumption does not match')

#Input variables (Marker)

M_d = 10                                # Average marker design time (minute)
z = 0.60                                # Printing speed per minute
vv = 0.42                                #Standard cost per minute in marker making floor (labor, machine & electricity)


#Input variables (Cutting Time)

T_G = 30                                  # General Preparation Time
xx= 9.908           # Average spreading speed in minutes after taking account for the idle strokes. 
T_M= 4           # Time for Placement of the marker
t_c= 4.73         # SMV of cutting time per garment pattern
T_S= 5           # preparation time for sticker placement
𝑡_𝑏 = 2.937                               # Standard minute value (SMV) of time takes to bundle.
𝑏 =  15                                   # pieces of garments in one bundle
ww = 0.30                                 # standard cost per minute in cutting floor (labor, machine & electricity)
P_min, P_max= 10,400

In [2]:
import numpy as np
import math
from copy import deepcopy
import plotly.express as px
import random
import time
rng = np.random.default_rng()
import pandas as pd
from itertools import product

In [3]:
def Shortage_allowance(Q,d=0.01):
    temp=np.dot((1-d),Q)
    return [round(i) for i in temp]
Q_b= Shortage_allowance(demand,d)


In [4]:
def Update_Res(R,GG,PP):
    for s in range(len(GG)): #Updating Residual within the while loop
        R=R-np.dot(GG[s],PP[s])
    return R

In [5]:
def Length(g_i_j):
    l_i = e+ np.dot(g_i_j,Y_b)/U
    return l_i

In [6]:
from Heuristics import H1,H3,H5,H3_MB2016,H1_T2020

## Crossover from (M'Hallah and Bouziri, 2016, p. 338)

In [7]:
def Crossover(Parent_1,Parent_2,crossover_rate=1):
    '''
    Parent_1,Parent_2 are dictionary containing G_a_b & P_a as values    
    '''
    
    if rng.random() > crossover_rate:
        return Parent_1

    
    G1 = Parent_1['G']
    P1 = Parent_1['P'] 
    G2 = Parent_2['G']
    P2 = Parent_2['P']
    #print('P1:',P1)

    m_1=len(G1)
    Beta=len(G1[0])
    
    
    cp=rng.integers(0,m_1,size=2)
    cp.sort()    #cp is the cutpoint for crossover

    
    C_G= G1[cp[0]:cp[1]+1]
    C_P= P1[cp[0]:cp[1]+1]


    '''a few section is taken from Parent 1'''
        

    
    r_b= Update_Res(Q_b, C_G, C_P)

    
    '''r_b is the residual demand after few section taken from Parent1'''
    
    i=0
    ''' i is the index number of sections(G) from Parent_2'''
    
    while max(r_b)>0:
        
        g_i_j= list (np.zeros(Beta,dtype=int))
        
        if i<=(len(P2)-1) and max(r_b)>P_min:     
            
            K=[]
            for j in range(Beta):
                if G2[i][j]>0:
                    K.append(j)
            '''K in the list of index where g_i_j of Parent2 is greater than 0'''
            
            #print(f'G2: {G2} and K:{K}')
            try:
                GCD= np.gcd.reduce([r_b[k] for k in K if r_b[k]>0])
                
            except:  #if the list is null
                GCD=1 
            '''GCD is LCD (r_s : r_s > 0, O_i_s > 0, s ∈ S)  
            from (M'Hallah and Bouziri, 2016, p. 338)'''

            p_i = max(P_min,min(P2[i],GCD)) ###
            
            #print(f'-{p_i}-')
            for j in K:
                '''K in the list of index where g_i_j of Parent2 is greater than 0'''
                if r_b[j]>0:
                    g_i_j[j]=min(math.floor(r_b[j]/p_i),math.floor((l_max-Length(g_i_j))/(Y_b[j]/U)))
                '''else 0'''
            
            r_b= r_b-np.dot(g_i_j,p_i)
     
            if max(g_i_j)>0:   #This is to avoid all zero section from adding
                C_G.append(g_i_j)
                C_P.append(p_i)
            i+=1    # goto the next section of Parent2
            
        else:
            p_i=min(P_max,max(P_min,np.gcd.reduce([k for k in r_b if k>0])))
            #print(f'+{p_i}+')
            for j in range(Beta):
                if r_b[j]>0:
                    g_i_j[j]=min(math.ceil(r_b[j]/p_i), math.floor((l_max-Length(g_i_j))/(Y_b[j]/U)))
                
                '''else 0 is already there '''

            r_b= r_b-np.dot(g_i_j,p_i)
            
            if max(g_i_j)>0:
                C_G.append(g_i_j)
                C_P.append(p_i)
            i+=1
            #break
            
    return {'G':C_G,'P':C_P}

## Mutation Operator from M&B 2016

In [8]:
def mutate(ii,jj,G,P):    

    G[ii][jj]=0
    r = Update_Res(Q_b,G,P)
    Beta=len(G[0])
    
    if max(r)>0:
        II=[x for x in range(len(G)) if x != ii and P[ii] <= max(r)]

    while max(r)>0:
        if II:

            L={s: Length(G[s]) for s in II}
            min_L=min(L)
            idx= min(L, key=L.get)   #index to the min lengh of the sections 

            G[idx][jj]= G[idx][jj] + min(math.ceil(r[jj]/P[idx]), math.floor((l_max-Length(G[idx]))/(Y_b[jj]/U)))
            

            II.remove(idx)  #removing the updated section from II
        
        else:
            #print('create new section')
            g= list(np.zeros(Beta,dtype=int))
            p= min(P_max,max(P_min,r[jj]))
            if p> r[jj]:
                g[jj]=1
            else:
                g[jj]=min(math.floor(r[jj]/p), math.floor(l_max/(Y_b[jj]/U)))  
            G.append(g)
            P.append(p)
            
        
        r = Update_Res(Q_b,G,P)  #Updating Residual

    return G,P


def Mutation(Child, mutation_rate=0.05):

    temp_Ch=deepcopy(Child)
    M_G =temp_Ch['G']
    M_P =temp_Ch['P']
    Beta=len(M_G[0])
    
    for i in range(len(M_P)):
        for j in range(Beta):
            if rng.random()<mutation_rate:
                M_G,M_P = mutate(i,j,M_G,M_P)   #user defined fuction

    return {'G':M_G,'P':M_P}



## Objective Function

In [9]:
def ObjectiveFunction (chromosome):
    
    G_a_b = chromosome['G']
    P_a = chromosome['P']
    Alpha = len(P_a)             # number of Sections
    

    '''                         Fabric Cost                      '''
        
    # Total fabric length = L # Total Fabric Cost = C_F

    l_a=[Length(G_a_b[alpha]) for alpha in range(Alpha) ] #Length function
    L= np.dot(l_a,P_a)       #Multiply then Sum
    C_F = L*f
    #print('Total Fabric Cost = C_F: ',C_F)
    
    
    '''                        Marker Cost                        '''
    
    #Marker Making Cost = C_M

    M_p_a = [(la-e)/z for la in l_a]     # devide each element of a 'l_a' by 'z'
    #M_p_a = Marker Printing time (minute) of section alpha 
 
    '''
    𝑟 = {1 ; 𝑖𝑓 𝑡h𝑒 𝑚𝑎𝑟𝑘𝑒𝑟 𝑖𝑠 𝑏𝑒𝑖𝑛𝑔 𝑢𝑠𝑒𝑑 𝑓𝑜𝑟 𝑡h𝑒 𝑓𝑖𝑟𝑠𝑡 𝑡𝑖𝑚𝑒 
        {0 ; 𝑖𝑓 𝑡h𝑒 𝑚𝑎𝑟𝑘𝑒𝑟 h𝑎𝑠 𝑏𝑒𝑒𝑛 𝑢𝑠𝑒𝑑 𝑏𝑒𝑓𝑜𝑟𝑒
    '''
    r=[]
    for i in range(Alpha):
        temp=0
        j=i-1
        while j>=0:
            if G_a_b[i]== G_a_b[j]:
                temp+=1
                break
            j-=1
        if temp==0:
            r.append(1)
        else:
            r.append(0)

    
    C_M = 0
    for α in range(Alpha):
        if l_a[α]>e:   # this makes sure that section has at least one garments 
            C_M += (M_d*r[α] + M_p_a[α])*vv       
    
    # 'if la>e' makes sure that the section contain at least one garments, 
    #  not all G_a_b values are zero
    
    
    '''                          Cutting Cost                            '''
        
    # Cutting Time of one section = T_T  # Total Cutting Cost = C_C
    
    #T_T  =T_G + T_F +T_M+ T_c+T_S +T_B
    
    T_C=[]   #Cutting time for every section
    for alpha in range(Alpha):
        T_C.append(sum(G_a_b[alpha])*t_c)
    
    T_F=[]  # Fab spreading time for each section
    for α in range(Alpha):
        T_F.append(l_a[α]*P_a[α]/xx)
    
    T_B=[] #Bundleing time for each section
    for α in range(Alpha):
        T_B.append(math.ceil(P_a[α]/b)*sum(G_a_b[α])*t_b)
    
    
    T_T_T = 0  #Total cutting time
    for α in range(Alpha):
        if l_a[α]>e:   # this makes sure that section has at least one garments
            T_T_T+=T_G+T_F[α]+T_M+T_C[α]+T_S+ T_B[α]
    
    
    C_C = T_T_T*ww  #Total cutting cost
    
    
    
    '''                              Total Cost                 '''
    # Total Cost = C_T = C_F + C_M + C_C
    
    return C_F+C_M+C_C

## Fitness Score

In [10]:
def Fitness(chromosome): 
  
    G_a_b= chromosome['G']
    P_a = chromosome['P']
    Beta= len(G_a_b[0])
    
    score= ObjectiveFunction(chromosome)
    #print('score:',score)
    fitness_score=score
    
                
    '''       Penalty for shortage production           '''
    R= Update_Res(R=demand,GG=G_a_b,PP=P_a)
    for beta in range(Beta):
        if R[beta]>0:
            s_penalty= R[beta]/sum(demand)
            fitness_score +=score*s_penalty/2 
    
    
    '''         Penalty for excess production           '''
    r=np.dot(1.02,demand)         # additional 2% allowance
    R= Update_Res(R=r,GG=G_a_b,PP=P_a)
    #print(R)
    for beta in range(Beta):
        if R[beta]<0:
            e_penalty= (-R[beta]/sum(demand))   # 2times than s_penalty
            fitness_score +=score*e_penalty   
            
    
    '''       double check if the solution is valid       '''
    res= Update_Res(R=Q_b,GG=G_a_b,PP=P_a)
    if max(res)>0:
        print('solution is unvalid in demand constraint')
        fitness_score +=10000   #this will eventualy make the solution extinct.

    for i in G_a_b:
        if Length(i)>l_max:
            print('breaking !! fitness !! solution is not valid in length constraint: ',i, Y_b, l_max,Length(i))
            fitness_score +=10000
    
    return fitness_score

# Fitness(Sol_1)    

## Function Initial Population Generation

In [11]:
def GeneratePopulation(pop_size):
    sol1=H3_MB2016(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,l_max=l_max,e=e)
    sol2=H1_T2020(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,l_max=l_max,e=e)
    P_of_S=[]
    for sol in (sol1,sol2):
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    
    while len(P_of_S)<pop_size:
        h=rng.integers(0,3)
        #print('h:',h)
        if h==0:
            sol= H1(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        elif h==1:
            sol=H3(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        else:
            sol=H5(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    return P_of_S
#Pool_of_Sol= GeneratePopulation(50)
#len(Pool_of_Sol)

In [12]:
def S_with_F(p_o_s):
    for i in range(len(p_o_s)): 
        if 'F' not in p_o_s[i]:
            p_o_s[i]['F']=Fitness(p_o_s[i])
    return p_o_s

In [13]:
# P_of_S_with_F= S_with_F(Pool_of_Sol)
# print(type(P_of_S_with_F))

## Tournament Selection

In [14]:
def TournamentSelection(sol_list, k=3):
    '''
    k= number of random parents that take part in one tournament
    '''

    indexes= rng.integers(0,len(sol_list), size=k) 
    
    psbl_parents = [sol_list[i] for i in indexes]
    
    psbl_parents = sorted(psbl_parents, key=lambda d: d['F'])
    return deepcopy(psbl_parents[0])


## GA with TS

In [16]:
def GA_with_TS(pop_size, generation,k=3,m_rate=0.05,c_rate=0.95):
    
    pool_of_sol= GeneratePopulation(pop_size)
    pool_of_sol= S_with_F(pool_of_sol)
    bests=[]
    sc=0
    for g in range(generation):
        for p in range(pop_size):
            Parent1,Parent2= [TournamentSelection(pool_of_sol,k=k)for i in range(2)]
            child = Crossover(Parent1, Parent2, crossover_rate=c_rate)
            mutated_child=Mutation(child,mutation_rate=m_rate)
            mutated_child['F']=Fitness(mutated_child)
            pool_of_sol.append(mutated_child)
        
        pool_of_sol= sorted(pool_of_sol, key=lambda d: d['F'])
        pool_of_sol = pool_of_sol[:pop_size]
        
        if bests:
            if bests[-1]['F']> pool_of_sol[0]['F']:
                sc=g
        bests.append(pool_of_sol[0])
        

    # xx=[i for i in range(len(bests))]
    # yy=[bests[i]['F'] for i in range(len(bests))]
    # fig=px.line(x=xx,
    #             y=yy,
    #             title=f'pop_size={pop_size},generation= {generation},k={k},mu_rate= {m_rate},c_rate={c_rate}, best={yy[-1]}, sc={sc}',
    #             labels=dict(x='generation',y='fitness'))
    # fig.show()
    
    return bests[-1],sc

In [17]:
# GA_with_TS(pop_size=100, generation=100)

## Parameter Tuning

In [18]:
from itertools import product
k=[2,3,4,5,6]
c_rate=[0.8,0.9,0.95,1]
m_rate=[0.003,0.01,0.05,0.1]

iteration=product(k,c_rate,m_rate)
df = pd.DataFrame(columns=['demand','sol','k','c_rate','m_rate','Fitness','Runtime','sc'])
for k,c_rate,m_rate in iteration:
    Dataset={ 'demands':[[10,21,33,41,39,40,30],
                         [193,501,1018,1249,998,564,250,128],
                         [990,1980,3961,2971,1980]],
             'consumption':[[1.5073,1.5685,1.6126,1.6704,1.7182,1.7911,1.8308],
                            [1.3350,1.3998,1.4356,1.4826,1.5440,1.5878,1.6313,1.6908],
                            [1.3086,1.3671,1.4183,1.4538,1.5122]],
             'price':[3.10,2.90,1.95]
             }
    for i in range(len(Dataset['demands'])):
        demand=Dataset['demands'][i]
        Q_b= Shortage_allowance(demand,d)
        Y_b=Dataset['consumption'][i]
        f=Dataset['price'][i]
        for j in range(5):
            time_start = time.perf_counter()
            Gbest,sc = GA_with_TS(pop_size=200, generation=180,k=k,m_rate=m_rate,c_rate=c_rate)
            time_elapsed = (time.perf_counter() - time_start)
            df=df.append({'demand':demand,'k':k,'sol':Gbest,'c_rate':c_rate,'m_rate': m_rate, 
                          'Fitness':Fitness(Gbest), 'Runtime': time_elapsed,'sc':sc}, 
                         ignore_index=True)
    #df.to_csv('GA_GridSearch_All Data_7_jan_23.csv')

KeyboardInterrupt: 

## GA with Stopping Criteria

In [20]:
def GA_with_TS_with_sc(pop_size, generation,k=3,m_rate=0.05,c_rate=0.95, sc=100):
    
    pool_of_sol= GeneratePopulation(pop_size)
    pool_of_sol= S_with_F(pool_of_sol)

    bests=[]
    it=0
    for g in range(generation):
        for p in range(pop_size):            
            Parent1,Parent2= [TournamentSelection(pool_of_sol,k=k)for i in range(2)]
                    
            child = Crossover(Parent1, Parent2, crossover_rate=c_rate)

            mutated_child=Mutation(child,mutation_rate=m_rate)
            mutated_child['F']=Fitness(mutated_child)
            pool_of_sol.append(mutated_child)
        
        pool_of_sol= sorted(pool_of_sol, key=lambda d: d['F'])
        pool_of_sol = pool_of_sol[:pop_size]
        
        if bests:
            if bests[-1]['F']> pool_of_sol[0]['F']:
                it=0
            else: 
                it+=1
        bests.append(pool_of_sol[0])
        if it> sc:
            print('break')
            break
        

#     x=[i for i in range(len(bests))]
#     yy=[bests[i]['F'] for i in range(len(bests))]
#     fig=px.line(x=x,
#                 y=yy,
#                 title=f'pop_size={pop_size},generation= {generation},k={k},mu_rate= {m_rate},c_rate={c_rate}, best={yy[-1]}, sc={sc}',
#                 labels=dict(x='generation',y='fitness'))
#     fig.show()
    
    return bests[-1]

### Parameter tuning for Popsize and Generartion

In [21]:
pop_size=[160,180,200,250]
generation=[160,180,200,250]
df = pd.DataFrame(columns=['demand','sol','pop_size','generation','Fitness','Runtime'])
for pop_size,generation in product(pop_size,generation):
    Dataset={ 'demands':[[10,21,33,41,39,40,30],
                         [193,501,1018,1249,998,564,250,128],
                         [990,1980,3961,2971,1980]],
             'consumption':[[1.5073,1.5685,1.6126,1.6704,1.7182,1.7911,1.8308],
                            [1.3350,1.3998,1.4356,1.4826,1.5440,1.5878,1.6313,1.6908],
                            [1.3086,1.3671,1.4183,1.4538,1.5122]],
             'price':[3.10,2.90,1.95]
             }
    
    
    k, c_rate, m_rate,sc = (2,0.95, 0.003,99) #parameters
    for i in range(len(Dataset['demands'])):
        demand=Dataset['demands'][i]
        Q_b= Shortage_allowance(demand,d)
        Y_b=Dataset['consumption'][i]
        f=Dataset['price'][i]
        for j in range(3):
            time_start = time.perf_counter()
            Gbest= GA_with_TS_with_sc(pop_size=pop_size, generation=generation,k=k,m_rate=m_rate,c_rate=c_rate, sc=sc)
            time_elapsed = (time.perf_counter() - time_start)
            
            df=df.append({'demand':demand,'sol':Gbest,'pop_size':pop_size ,'generation':generation,
                          'Fitness':Fitness(Gbest), 'Runtime': time_elapsed}, 
                         ignore_index=True)
    #df.to_csv('GA_pop_generation_tuning_7_jan_23.csv')

break
break
break
break
break


KeyboardInterrupt: 

## Selected Parameters : 
* k= 2, 
* c_rate= 0.95, 
* m_rate= 0.003, 
* sc= 99 (stoping criteria)
* pop_size= 200, 
* generation= 200

## Implemanting on Industry Data

In [20]:
path='/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/DataSets.xlsx'

In [21]:
df=pd.read_excel(path,sheet_name='Sheet3',index_col=[0])

In [22]:
df=df.T   #transposing the datatable

df.columns = df.columns.str.strip()

col=list(df.columns)  #taking the list of columns or 'styles'
col= sorted(set(col), key=col.index)


In [23]:
df

Style short,ST1,ST1,ST1,ST1,ST1,ST1,ST1,ST1,ST2,ST2,...,ST34,ST34,ST35,ST35,ST35,ST35,ST35,ST35,ST35,ST35
Style,Girl's High Rise Wide Leg Belted Trouser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Falcon lake shorty short,NaN,...,NaN,NaN,Mens Pull on Jogger,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Demand,136,354,743,928,738,423,188,91,872,1743,...,48,34,0,10,21,33,41,39,40,30
Consumption,2.0581,2.128,2.2061,2.2688,2.3334,2.4134,2.4793,2.597,0.612,0.6315,...,1.6377,1.7032,1.4688,1.5073,1.5685,1.6126,1.6704,1.7182,1.7911,1.8308
Price/unit length,3.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.51,NaN,...,NaN,NaN,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
col

['ST1',
 'ST2',
 'ST3',
 'ST4',
 'ST5',
 'ST6',
 'ST7',
 'ST8',
 'ST9',
 'ST10',
 'ST11',
 'ST12',
 'ST13',
 'ST14',
 'ST15',
 'ST16',
 'ST17',
 'ST18',
 'ST19',
 'ST20',
 'ST21',
 'ST22',
 'ST23',
 'ST24',
 'ST25',
 'ST26',
 'ST27',
 'ST28',
 'ST29',
 'ST30',
 'ST31',
 'ST32',
 'ST33',
 'ST34',
 'ST35']

In [27]:
df_result_GA = pd.DataFrame(columns=['style','demand','sol','Fitness','Cost','Runtime','P_cost','P_time'])
k, c_rate, m_rate,sc = (2,0.95,0.003,99) #parameters
pop_size,generation=(200,200)
for j in range(5):
    for i in col:
        demand=df[i].loc['Demand'].to_list()
        demand=list(map(int,demand))
        #print('demand=',demand)

        d=0  #shortange allowance in Percentage
        Q_b= Shortage_allowance(demand,d)

        consumption=df[i].loc['Consumption'].to_list()
        Y_b=list(map(float,consumption))

        if len(Q_b)!=len(Y_b):
            raise ValueError('number of sizes and number of fabric consumption does not match')

        price=df[i].loc['Price/unit length'].to_list()
        price=list(map(float,price))
        f=price[0]

        print(i,demand,Q_b,Y_b,f)
        time_start = time.perf_counter()
        Gbest = GA_with_TS_with_sc(pop_size=pop_size, generation=generation,k=k,m_rate=m_rate,c_rate=c_rate, sc=sc)
        time_elapsed = (time.perf_counter() - time_start)
        print(Gbest['F'])
        df_result_GA=df_result_GA.append({'style':i,'demand':demand,'Cost': ObjectiveFunction(Gbest),'sol':Gbest,
                                          'Fitness':Gbest['F'],'Runtime': time_elapsed},
                                         ignore_index=True)
    #df_result_GA.to_excel('/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/GA_industry_data_7_jan_23.xlsx')

ST1 [136, 354, 743, 928, 738, 423, 188, 91] [136, 354, 743, 928, 738, 423, 188, 91] [2.0581, 2.128, 2.2061, 2.2688, 2.3334, 2.4134, 2.4793, 2.597] 3.56


KeyboardInterrupt: 

In [ ]:
df_result_GA